# Modellierung der Vorhersage der Prozessqualität


Dieses Notebook beschäftigt sich mit der Entwicklung von Vorhersagemodellen für die Zielgröße. Hierzu werden verschiedene Featuresets und Modelle verwendet, um die bestmögliche Vorhersagegenauigkeit zu erreichen.

## Einstellungen und Datenvorbereitung

- **Import relevanter Bibliotheken**
- **Metrik:** Die Leistung der Modelle wird anhand der `f1` bzw. `f1_weighted`-Metriken bewertet.
- **Zielgröße (Target):** `Ergebnis_con`
- **Irrelevante Spalten:** Diese Spalten werden aus dem Feature-Set entfernt, da sie für die Vorhersage nicht benötigt werden.

In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('..'))

from sklearn.model_selection import train_test_split, StratifiedKFold
from IPython.display import display, HTML
from helpers.model_utils import (load_and_prepare_data, 
                                 display_top_models_for_target, 
                                 train_and_tune_models_balanced,  
                                 compare_results_across_datasets, 
                                 display_final_confusion_matrices, 
                                 save_scores_to_csv, 
                                 analyze_model_performance, 
                                 filter_and_retrain_with_vif,
                                 direcetion_results,
                                 plot_scores_and_percent_diff,
                                 compare_smote_effects)
from helpers.model_pipelines import (define_pipelines_Prozessqualitaet, 
                                     shap_analysis,load_and_split_data,
                                     shap_analysis_single_model)

In [ ]:
# Einstellungen Notebook
target_name = "Prozessqualität"

# Metrik auswählen
metric = "f1_weighted"  

# Zielgröße
target_column = 'Ergebnis_con'  
irrelevant_columns = ['Material_con', 'Position_con', 'Probenhoehe', 'richtig_verbaut', 'Zeit', 'VersuchID','umformzeit', 'synthetisch'
                      ]

# Einfluss der Verkippungssensoren
verkippungsfeatures = ["Verkippung_1", "Verkippung_2", "Verkippung_3", "Verkippung_4", 
                      "Verkippung_1_Min", "Verkippung_1_Max", "Verkippung_1_Mean", "Verkippung_1_Median", "Verkippung_1_Std", 
                      "Verkippung_2_Min","Verkippung_2_Max","Verkippung_2_Mean","Verkippung_2_Median","Verkippung_2_Std","Verkippung_3_Min",
                      "Verkippung_3_Max","Verkippung_3_Mean","Verkippung_3_Median","Verkippung_3_Std","Verkippung_4_Min","Verkippung_4_Max",
                      "Verkippung_4_Mean","Verkippung_4_Median","Verkippung_4_Std", "tilt_x_tiefster", "tilt_y_tiefster", "tilt_x_t0", "tilt_y_t0"]

# Mapping Pfad für Labels
label_mapping_path = "../data_preparation/mappings/label_mappings_binary.json"

# Verkippung berücksichtigen?
Verkippung = True
if not Verkippung:
    irrelevant_columns.extend(verkippungsfeatures)
    
# SHAP Analyse für ALLE Modelle durchführen?
shap_on = False
plot_type = 'summary'  # Alternativ: "summary", "bar" oder "interaction" oder "violin"
plot_type = str(plot_type)

# Plots speichern?
save_plots = True

# mit SMOTE (synthethische Erweiterung der Trainingsdaten)?
smote_on = False

# Bautiel Temperatur berücksichtigen?
bauteil_temp = True
if not bauteil_temp:
    irrelevant_columns.append('Bauteil_Temp')

In [ ]:
# Pfade zu den balancierten Testdaten
test_filepath = f"../datasets/test_{target_column}.pkl"
if smote_on:
    # Pfade zu den balancierten Trainingsdaten
    train_filepath = f"../datasets/train_balanced_{target_column}_smote_2500.pkl"
    test_filepath = f"../datasets/test_{target_column}.pkl"
    # Lade und bereite Trainings- und Testdaten vor
    X_train_original, y_train = load_and_prepare_data(train_filepath, target_column, irrelevant_columns)
    X_test_original, y_test = load_and_prepare_data(test_filepath, target_column, irrelevant_columns)
else: 
    filepath = f"../datasets/new_features.pkl"
    X, y = load_and_split_data(filepath, target_column, irrelevant_columns)
    # Datenaufteilung
    X_train_original, X_test_original, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    
# Überprüfen der Shapes
print(f"Trainingsdaten: X_train={X_train_original.shape}, y_train={y_train.shape}")
print(f"Testdaten: X_test={X_test_original.shape}, y_test={y_test.shape}")

# Zentrale Pipeline und Parameter
target_pipeline = define_pipelines_Prozessqualitaet()
pipelines, param_grids = target_pipeline

# Cross-Validation Setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Anzeige der Klassenverteilungen 
print("Klassenverteilung im Training:")
print(y_train.value_counts())
print("\nKlassenverteilung im Test:")
print(y_test.value_counts())

In [ ]:
print(X_train_original.columns)
print(len(X_train_original.columns))

# Process Features Dataset

In diesem Abschnitt wird das Datenset für die Prozessmerkmale vorbereitet und trainiert. Das Dataset wird angepasst, indem irrelevante Spalten entfernt und die relevanten Features extrahiert werden. Anschließend werden verschiedene Machine-Learning-Modelle trainiert, die darauf abzielen, die Zielgröße basierend auf den Prozessmerkmalen vorherzusagen.

In [ ]:
# Einstellungen Dataset
dataset_name = "process_features"

target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)

# Auswahl der Process Features aus dem Dataset
columns_process_features = [
    "VersuchID",
    "Berührzeit",
    "Höhe_Wegmessung_Aufprall",
    "Umformzeit",
    "Tiefster_Punkt",
    "Energie_Aufprall",
    "Motorstrom_Durchschnitt",
    "Energie_ab_Durchschnitt",
    "Bauteil_Temp",
    "Werkzeug_Temp",
    "Material_con",
    "Position_con",
    "Ergebnis_con",
    "Probenhoehe",
]

# Sicherstellen, dass nur existierende Spalten verwendet werden
columns_to_keep_train = [col for col in columns_process_features if col in X_train_original.columns]

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original[columns_to_keep_train]
X_test = X_test_original[columns_to_keep_train]

# Falls Fehler auftreten, können wir sicherstellen, dass alle Spalten korrekt bereinigt sind:
X_train.columns = X_train.columns.str.strip()  # Entferne führende und nachfolgende Leerzeichen
X_test.columns = X_test.columns.str.strip()

print("Bereinigte Spalten in X_train:", X_train.columns)
print("Bereinigte Spalten in X_test:", X_test.columns)
print(len(X_train.columns))

In [ ]:
target_dir

## Training

In diesem Abschnitt wird das Training der Modelle durchgeführt. Die Modelle werden mit Hilfe von Cross-Validation optimiert, um eine robuste Bewertung der Modellleistung sicherzustellen. Hyperparameter-Tuning wird genutzt, um die besten Einstellungen für jedes Modell zu finden und die Generalisierungsfähigkeit auf Testdaten zu verbessern.

In [ ]:
# Modelltraining und Bewertung
best_pipelines, results, confusion_matrices1 = train_and_tune_models_balanced(
    pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    skf, 
    X_test=X_test, 
    y_test=y_test
)

# Ergebnisse in DataFrame speichern
results_df_1 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_1.to_html()))

#results_df_1.drop(['best_params'], axis=0, inplace=True)
# Ergebnisse speichern
save_scores_to_csv(
    results=results_df_1,  # Direkt das DataFrame übergeben
    output_dir=target_dir,    # Zielverzeichnis
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Dateiname
    Verkippung=Verkippung    # Flag für Verkippung
)

## Confusion Matrices

In diesem Abschnitt werden die Confusion-Matrices der Modelle analysiert. Diese bieten eine detaillierte Übersicht über die Vorhersagegenauigkeit der Modelle und ermöglichen es, Fehlklassifikationen zu identifizieren. Durchschnittliche Confusion-Matrices über alle Cross-Validation-Folds werden erstellt, visualisiert und optional gespeichert, um die Klassifikationsleistung der Modelle besser zu bewerten.

In [ ]:
# Normalisierte durchschnittliche Confusion-Matrices anzeigen
display_final_confusion_matrices(
    confusion_matrices1,
    label_mapping_path=label_mapping_path,
    target_column=target_column,  # Zielspalte hinzufügen
    target_dir=target_dir,
    save_plots=save_plots,
    normalize=True,  # Optional
    Verkippung=Verkippung
)

## Overfitting Test

In diesem Abschnitt wird die Überprüfung von Overfitting durch die Analyse von Learning Curves und Validation Curves durchgeführt, um den Bias-Variance-Tradeoff besser zu verstehen. Die `analyze_model_performance` Funktion aus der `model_pipelines.py` führt diese Analysen durch und erstellt entsprechende Plots. Es werden sowohl die Trainingsscores als auch die Testscores über verschiedene Größen des Trainingssets und Hyperparameterbereiche betrachtet, um die Modellstabilität und die Generalisierungsfähigkeit zu bewerten. Dies hilft dabei, die optimale Komplexität der Modelle zu bestimmen und sicherzustellen, dass sie gut auf unbekannte Daten generalisieren.

In [ ]:
# Overfitting überprüfen 
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring='f1', 
    cv=skf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)


## SHAP-Analyse

In diesem Abschnitt führen wir eine SHAP-Analyse durch, um die Einflüsse der einzelnen Features auf die Vorhersagen der besten Modelle zu verstehen. Die Ergebnisse dieser Analyse helfen uns, die Modellentscheidungen transparenter zu machen. Anschließend laden wir Diagramme, die die besten drei Modelle basierend auf unterschiedlichen Metriken darstellen. Diese Visualisierungen unterstützen die Bewertung der Modellperformance und die Identifizierung von Schlüsselfeatures, die die Zielgröße beeinflussen.

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
    best_pipelines=best_pipelines,  # Das Dictionary der besten Pipelines
    X_train=X_train,  # Die Trainingsdaten
    target_name = target_name,  # Der Zielname, z. B. für das Modellieren des Zielwerts
    dataset_name= dataset_name,  # Der Name des Datensets
    output_dir= target_dir,  # Optional: Basisverzeichnis für die Ergebnisse
    plot_type=plot_type,
    save_plots=save_plots,  # Angabe, ob die Diagramme gespeichert werden sollen
    verkippung = Verkippung
)


In [ ]:
# if shap_on:
#     feature_importances, irrelevant_features = extract_feature_importance_from_shap(
#         best_pipelines=best_pipelines,
#         X_train=X_train,
#         target_name=target_name,
#         dataset_name=dataset_name,
#         output_dir=target_dir,
#         verkippung=Verkippung,
#         importance_threshold=0.01,
#         save_results=save_plots
#     )

#     # Ausgabe der irrelevanten Features
#     print("Irrelevante Features:", irrelevant_features)

# Aggregierte Daten

In [ ]:
# Einstellungen
dataset_name = "aggregated_features"

target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)

# Auswahl der Process Features aus dem Dataset
columns_aggregated_features = [
    "VersuchID",
    "Berührzeit",
    "Höhe_Wegmessung_Aufprall",
    "Umformzeit",
    "Tiefster_Punkt",
    "Energie_Aufprall",
    "Motorstrom_Durchschnitt",
    "Energie_ab_Durchschnitt",
    "Bauteil_Temp",
    "Werkzeug_Temp",
    "Material_con",
    "Position_con",
    "Ergebnis_con",
    "Probenhoehe",
    "richtig_verbaut",
    "Wegmessung_Min",
    "Wegmessung_Max",
    "Wegmessung_Mean",
    "Wegmessung_Median",
    "Wegmessung_Std",
    "Verkippung_1_Min",
    "Verkippung_1_Max",
    "Verkippung_1_Mean",
    "Verkippung_1_Median",
    "Verkippung_1_Std",
    "Verkippung_2_Min",
    "Verkippung_2_Max",
    "Verkippung_2_Mean",
    "Verkippung_2_Median",
    "Verkippung_2_Std",
    "Verkippung_3_Min",
    "Verkippung_3_Max",
    "Verkippung_3_Mean",
    "Verkippung_3_Median",
    "Verkippung_3_Std",
    "Verkippung_4_Min",
    "Verkippung_4_Max",
    "Verkippung_4_Mean",
    "Verkippung_4_Median",
    "Verkippung_4_Std",
    "Stoesselhub_Min",
    "Stoesselhub_Max",
    "Stoesselhub_Mean",
    "Stoesselhub_Median",
    "Stoesselhub_Std",
    "Geschwindigkeit_Ges_Min",
    "Geschwindigkeit_Ges_Max",
    "Geschwindigkeit_Ges_Mean",
    "Geschwindigkeit_Ges_Median",
    "Geschwindigkeit_Ges_Std",
    "Presskraft_dyn_Min",
    "Presskraft_dyn_Max",
    "Presskraft_dyn_Mean",
    "Presskraft_dyn_Median",
    "Presskraft_dyn_Std",
    "Motorstrom_Min",
    "Motorstrom_Max",
    "Motorstrom_Mean",
    "Motorstrom_Median",
    "Motorstrom_Std",
]

# Sicherstellen, dass nur existierende Spalten verwendet werden
columns_to_keep_train = [col for col in columns_aggregated_features if col in X_train_original.columns]

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original[columns_to_keep_train]
X_test = X_test_original[columns_to_keep_train]

# Falls Fehler auftreten, können wir sicherstellen, dass alle Spalten korrekt bereinigt sind:
X_train.columns = X_train.columns.str.strip()  # Entferne führende und nachfolgende Leerzeichen
X_test.columns = X_test.columns.str.strip()

print("Bereinigte Spalten in X_train:", X_train.columns)
print("Bereinigte Spalten in X_test:", X_test.columns)
print(len(X_train.columns))

## Training

In [ ]:
# Modelltraining und Bewertung
best_pipelines, results, confusion_matrices2 = train_and_tune_models_balanced(pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    skf, 
    X_test=X_test, 
    y_test=y_test
)

# Ergebnisse in DataFrame speichern
results_df_2 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_2.to_html()))

save_scores_to_csv(
    results=results_df_2,  # DataFrame mit den Scores
    output_dir=target_dir,
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Gemeinsamer Basisname
    Verkippung=Verkippung  # Fügt den '_no_tilt'-Suffix hinzu, falls nötig
)

## Confusion Matrices

In [ ]:
# Normalisierte durchschnittliche Confusion-Matrices anzeigen
display_final_confusion_matrices(
    confusion_matrices2,
    label_mapping_path=label_mapping_path,
    target_column=target_column,  # Zielspalte hinzufügen
    target_dir=target_dir,
    save_plots=save_plots,
    normalize=True,  # Optional
    Verkippung=Verkippung
)

## Overfitting Test

In [ ]:
# Overfitting überprüfen 
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring='f1', 
    cv=skf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)

## SHAP-Analyse

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
    best_pipelines=best_pipelines,  # Das Dictionary der besten Pipelines
    X_train=X_train,  # Die Trainingsdaten
    target_name = target_name,  # Der Zielname, z. B. für das Modellieren des Zielwerts
    dataset_name= dataset_name,  # Der Name des Datensets
    output_dir= target_dir,  # Optional: Basisverzeichnis für die Ergebnisse
    plot_type=plot_type,
    save_plots=save_plots,  # Angabe, ob die Diagramme gespeichert werden sollen
    verkippung = Verkippung
)
    #starting_shap

# New Features Dataset

In [ ]:
# Einstellungen
dataset_name = "new_features"

target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original
X_test = X_test_original

## Training

In [ ]:
# Modelltraining und Bewertung
best_pipelines, results, confusion_matrices3 = train_and_tune_models_balanced(pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    skf, 
    X_test=X_test, 
    y_test=y_test
)

# Ergebnisse in DataFrame speichern
results_df_3 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_3.to_html()))

save_scores_to_csv(
    results=results_df_3,  # DataFrame mit den Scores
    output_dir=target_dir,
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Gemeinsamer Basisname
    Verkippung=Verkippung  # Fügt den '_no_tilt'-Suffix hinzu, falls nötig
)

## Confusion Matrices

In [ ]:
# Normalisierte durchschnittliche Confusion-Matrices anzeigen
display_final_confusion_matrices(
    confusion_matrices3,
    label_mapping_path=label_mapping_path,
    target_column=target_column,  # Zielspalte hinzufügen
    target_dir=target_dir,
    save_plots=save_plots,
    normalize=True,  # Optional
    Verkippung=Verkippung
)

## Overfitting Test

In [ ]:
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring='f1', 
    cv=skf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)

## SHAP-Analyse

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
    best_pipelines=best_pipelines,  # Das Dictionary der besten Pipelines
    X_train=X_train,  # Die Trainingsdaten
    target_name = target_name,  # Der Zielname, z. B. für das Modellieren des Zielwerts
    dataset_name= dataset_name,  # Der Name des Datensets
    output_dir= target_dir,  # Optional: Basisverzeichnis für die Ergebnisse
    plot_type=plot_type,
    save_plots=save_plots,  # Angabe, ob die Diagramme gespeichert werden sollen
    verkippung = Verkippung
)



# Vergleich der Results 


In [ ]:
results_dfs = {
    "Process Features": results_df_1,
    "Aggregated Features": results_df_2,
    "New Features": results_df_3
}

# Vergleich der Ergebnisse über die Datasets hinweg
compare_metric = "f1_weighted"
comparison_table, best_combination = compare_results_across_datasets(results_dfs, metric="f1")

# Vergleichstabelle in HTML-Format konvertieren und anzeigen
print(f"Vergleich der {compare_metric}-Scores für die unterschiedlichen Datasets und Modelle:")
display(HTML(comparison_table.to_html()))

# Beste Kombination ausgeben
print(f"\nBeste Kombination:")
print(f"Dataset: {best_combination[0]}, Modell: {best_combination[1]}, Score: {best_combination[2]:.4f}")

save_scores_to_csv(
    results=comparison_table,
    output_dir=target_dir,
    file_name=f"compare_model_scores_{target_name}_{compare_metric}.csv",
    Verkippung=Verkippung
)

In [ ]:
output_path = f"../results/{target_name}/{balance_suffix}{tilt_suffix}{temp_suffix}/compare_model_scores_{target_name}_{compare_metric}{balance_suffix}{tilt_suffix}{temp_suffix}.svg"

# Diagramm erstellen
plot_scores_and_percent_diff(results_dfs, target_name=target_name, output_path=output_path, y_lim=(0.7, 1.0), metric=metric)

# Reduktion der Features & Performance des Best Models
- Das beste Modell wird nach der Feature Reduktion mit Hilfe der SHAP Values im Zielverzeichis gespeichert.

In [ ]:
# Modell und Feature Datensatz auswählen
model_name = "kNN"
pipeline = best_pipelines[model_name]
dataset_name = "new_features"

target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix  = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp, model_name=model_name)

# Verzeichnis erstellen, falls es nicht existiert
os.makedirs(target_dir, exist_ok=True)

In [ ]:
# SHAP-Analyse für ein einzelnes Modell
# shap_analysis_single_model(
#     model_name=model_name,
#     pipeline=pipeline,
#     X_train=X_train,
#     target_name=target_name,
#     dataset_name=dataset_name,
#     output_dir=target_dir_best,
#     plot_type=plot_type,
#     save_plots=save_plots,
#     verkippung=Verkippung
# )

In [ ]:
#feature_importance_path_best = f"results/best_models/{target_name}/feature_importance_{model_name}.csv"

In [ ]:
# **AUFRUF DER FUNKTION**
best_pipelines, retrain_results = filter_and_retrain_with_vif(
    model_name=model_name,
    pipeline=pipeline,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    feature_importance_path=feature_importance_path_best,
    param_grids=param_grids,
    cv=skf,
    scoring=metric,
    pareto_split=0.8,  
    correlation_threshold=0.9,  
    vif_threshold=200.0,  
    output_dir=target_dir_best,
    target_name=target_name,
    dataset_name=dataset_name,
    verkippung=Verkippung,
    is_regression=False,
    label_mapping_path=label_mapping_path,    # Neuer Parameter für Confusion-Matrix
    target_column=target_column          # Neuer Parameter für Confusion-Matrix
)

# Visualisierung der Ergebnisse


## SMOTE für Prozess Features

In [ ]:
smote_path = f"../results/{target_name}/{tilt_suffix}{temp_suffix}/process_features/model_scores_process_features.csv"
no_smote_path = f"../results/{target_name}/{balance_suffix}{tilt_suffix}{temp_suffix}/process_features/model_scores_process_featuresnoSMOTE.csv"
output_path = f"../results/{target_name}/compare_SMOTE_process_features_{target_name}_{compare_metric}.svg"
%config InlineBackend.figure_format = 'svg'
df_comp = compare_smote_effects(no_smote_path, smote_path, metric="f1", title=f"Vergleich der Fehlermetrik für das Prozess Feature-Set: No SMOTE vs. SMOTE", save_path=output_path)
